# CoronaVirus Covid19 India Lockdown 2.0

In [58]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

# Karnataka Coronavirus Data

In [59]:
url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Karnataka'
all_tables = pd.read_html(url)

# we  need 6 th table that is all_tables[5]
rawCases = all_tables[5]

In [60]:
# now we have to clean

rawCases['Date Announced']=rawCases['Date Announced'].apply(pd.to_datetime)
"""
Not change dtype for  Date Discharged/ Deceased column since most of the entries
are yet to filled. Patients yet to be discharged
"""
rawCases.drop(labels='Ref(s)',axis=1,inplace=True)
#Splitting Sex-Age column into two and droping the column
rawCases['Gender']=rawCases['Sex-Age'].str[0]
rawCases['Age']=rawCases['Sex-Age'].str[1:]
rawCases.drop(labels='Sex-Age',axis=1,inplace=True)
rawCases.Gender=rawCases['Gender'].str.replace('f','F')
# less than 1 year old baby age is in months. Setting to value 1
for i,rows in rawCases.Age.iteritems():
    if 'month' in rows:
        rawCases.at[i,'Age']=1
rawCases.Age=rawCases.Age.apply(pd.to_numeric)

In [68]:
#Merging districts Bengaluru and Bengaluru (native of Ananthapura, AP)
for i,rows in rawCases.Location.iteritems():
    if '(' in rows:
        district=rows.split(sep='(')[0].strip()
        rawCases.at[i,'Location']=district


In [99]:
genderGrp=rawCases.groupby(by='Gender').size().reset_index(name='count')
genderGrp=genderGrp.rename(columns={'count':'Confirmed'})


px.bar(
    genderGrp,
    x = "Gender",
    y = "Confirmed",
    title= "Confirmed Cases based on Gender", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=500
)

In [87]:
bins = [0,3,12,20,40,60,np.inf]
ranges = ['<3','3-12','12-20','20-40','40-60','60+']
rawCases['AgeRange']=pd.cut(rawCases.Age,bins,labels=ranges)
#ageGroup=rawCases.groupby(by='AgeRange')
#ageGroup.AgeRange.count().plot(kind='bar');

ageGroup=rawCases.groupby(by='AgeRange').size().reset_index(name='count')
ageGroup=ageGroup.rename(columns={'AgeRange':'Age Group','count':'Confirmed'})


px.bar(
    ageGroup,
    x = "Age Group",
    y = "Confirmed",
    title= "Confirmed Cases based on Age of Patients", # the axis names
    color_discrete_sequence=["purple"], 
    height=500,
    width=800
)


In [100]:
districtData=rawCases.groupby(by='Location').size().reset_index(name='count')

In [101]:
sortedDistrict=districtData.sort_values(by='count',ascending=False)
sortedDistrict=sortedDistrict.rename(columns={'Location':'Districts','count':'Confirmed'})

In [103]:
fig = px.scatter(sortedDistrict.head(), x="Districts", y="Confirmed", size="Confirmed", color="Districts",
               hover_name="Districts", size_max=60)
fig.update_layout(
    title=str(5) +" Worst hit districts in Karnataka",
    xaxis_title="Districts",
    yaxis_title="Confirmed Cases",
    width = 700
    )
fig.show();